In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-lite

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import models,layers
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/completeBhageeraPayload.csv')

In [ ]:
df.head(5)

,mlid,ax,ay,az,gx,gy,gz
0,Walking,-0.430176,0.611572,-0.747803,-17.633587,-55.404579,4.854962
1,Walking,-0.600586,0.668945,-0.920898,10.916031,88.320610,-46.167938
2,Walking,-0.390137,0.568115,-0.972168,-23.702290,9.648855,-15.167939
3,Walking,-1.108887,0.886475,-0.582275,70.381676,-85.572517,47.000000
4,Walking,-0.442139,0.665039,-0.820068,44.259541,-87.793892,49.152672


In [ ]:
df['mlid'].value_counts() #data is imbalanced

Rest        8141
Sleep       5753
Walking     5241
Running     3698
Standing    1757
Sitting      839
Name: mlid, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
yle = LabelEncoder()
df['mlid'] = yle.fit_transform(df['mlid'])

In [ ]:
df['mlid'].value_counts() 

0    8141
3    5753
5    5241
1    3698
4    1757
2     839
Name: mlid, dtype: int64

In [ ]:
X = df.drop('mlid', axis =1)
Y= df['mlid']

In [ ]:
X.head(5)

,ax,ay,az,gx,gy,gz
0,-0.430176,0.611572,-0.747803,-17.633587,-55.404579,4.854962
1,-0.600586,0.668945,-0.920898,10.916031,88.320610,-46.167938
2,-0.390137,0.568115,-0.972168,-23.702290,9.648855,-15.167939
3,-1.108887,0.886475,-0.582275,70.381676,-85.572517,47.000000
4,-0.442139,0.665039,-0.820068,44.259541,-87.793892,49.152672


In [ ]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state = 42)

In [ ]:
# resampling th data
X_resampled , Y_resampled = smk.fit_resample(X, Y)

In [ ]:
Y_resampled.value_counts() #data is now balanced

3    7935
2    7850
4    7830
1    7696
5    7483
0    7340
Name: mlid, dtype: int64

In [ ]:
# Splitting the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_resampled,Y_resampled, test_size=0.2, random_state=0)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
# Convert the labels to one-hot encoded vectors
num_classes = 6 #number of unique classes
y_train_encoded = to_categorical(Y_train, num_classes = num_classes)
y_test_encoded = to_categorical(Y_test, num_classes = num_classes)

In [ ]:
y_train_encoded

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
# Define the architecture of the ANN model
model = Sequential()
model.add(Dense(64, activation = 'relu')) # input_dim represents the number of input features
model.add(Dense(64, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))   # Output layer with softmax activation for multiclass classification

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model_history = model.fit(X_train, y_train_encoded, epochs = 50, batch_size = 32)

Epoch 1/50
1154/1154 [==============================] - 10s 4ms/step - loss: 1.1387 - accuracy: 0.5908
Epoch 2/50
1154/1154 [==============================] - 6s 5ms/step - loss: 0.7587 - accuracy: 0.7142
Epoch 3/50
1154/1154 [==============================] - 7s 6ms/step - loss: 0.6398 - accuracy: 0.7627
Epoch 4/50
1154/1154 [==============================] - 7s 6ms/step - loss: 0.5743 - accuracy: 0.7881
Epoch 5/50
1154/1154 [==============================] - 6s 5ms/step - loss: 0.5476 - accuracy: 0.7962
Epoch 6/50
1154/1154 [==============================] - 4s 3ms/step - loss: 0.5265 - accuracy: 0.8032
Epoch 7/50
1154/1154 [==============================] - 3s 3ms/step - loss: 0.5088 - accuracy: 0.8106
Epoch 8/50
1154/1154 [==============================] - 3s 3ms/step - loss: 0.4967 - accuracy: 0.8158
Epoch 9/50
1154/1154 [==============================] - 4s 4ms/step - loss: 0.4825 - accuracy: 0.8223
Epoch 10/50
1154/1154 [==============================] - 3s 3ms/step - loss: 0.47

In [ ]:
print(model_history.history.keys())

dict_keys(['loss', 'accuracy'])


In [ ]:
# Evaluate the model
loss , accuracy = model.evaluate(X_test, y_test_encoded)

289/289 [==============================] - 1s 2ms/step - loss: 0.3847 - accuracy: 0.8642


In [ ]:
model.save('/content/my_model')

In [ ]:
!ls

completeBhageeraPayload.csv  my_model  sample_data


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/my_model')

In [ ]:
converter.target_spec.supported_types = [tf.float32]

In [ ]:
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

In [ ]:
tflite_model = converter.convert()

In [ ]:
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
open("converted_model.tflite", "wb").write(tflite_model)

22384

In [ ]:
# Load the converted TFLite model
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
try:
    interpreter.allocate_tensors()
    print("Model converted to TFLite successfully!")
except ValueError as e:
    print("Error loading the TFLite model:", e)

Model converted to TFLite successfully!


In [ ]:
!ls

completeBhageeraPayload.csv  converted_model.tflite  my_model  sample_data


In [ ]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# Prepare input data (modify according to your model's input requirements)
input_data = np.random.rand(*input_details[0]['shape']).astype(np.float64)


In [ ]:
# Set input tensor data
interpreter.set_tensor(input_details[0]['index'], input_data)

In [ ]:
# Run inference
interpreter.invoke()

In [ ]:
# Get output tensor data
output_data = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
input_data

array([[0.13436062, 0.42874944, 0.09225087, 0.74122766, 0.78453627,
        0.35525819]])

In [ ]:
output_data

array([[0.91701776, 0.04313487, 0.00703303, 0.01263261, 0.00242828,
        0.01775349]], dtype=float32)